In [25]:
! pip install pandas numpy scikit-learn statsbombpy
! pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Ayudame a constuir esto Para construir una "huella dactilar" robusta, no buscamos la mayor cantidad de variables, sino las más discriminatorias. El objetivo es capturar el comportamiento de un jugador a través de diferentes facetas del juego. Basado en la literatura de análisis de roles y perfilado de jugadores, podemos agrupar las métricas en categorías que representen estas facetas.   Aquí te presento una selección curada de variables, todas disponibles en el endpoint player_season_stats de la API de StatsBomb, y la justificación de por qué cada una es esencial para construir la huella dactilar. Variables Esenciales para la Huella Dactilar del Jugador 1. Pases y Progresión del Balón Esta categoría define cómo un jugador contribuye a mover el balón por el campo. ¿Es un constructor de juego seguro, un pasador que rompe líneas o alguien que prefiere los pases largos? player_season_passing_ratio: El porcentaje de pases completados. Es una métrica base de eficiencia y seguridad en la posesión. player_season_pass_length: La longitud promedio de los pases. Distingue a los jugadores de pases cortos y control (estilo "Tiki-taka") de aquellos que buscan pases largos y directos.   player_season_lbp_completed_90 (Line-Breaking Passes): Pases completados que rompen una línea defensiva rival. Esta es una métrica crucial de los datos 360 y es fundamental para identificar a los mediocampistas y defensores de élite que crean oportunidades desde atrás.   player_season_op_passes_into_box_90: Pases completados desde juego abierto que terminan dentro del área rival. Mide la capacidad de un jugador para entregar el balón en la zona más peligrosa del campo. 2. Creación de Oportunidades y Finalización Aquí medimos la participación directa de un jugador en la generación y finalización de jugadas de gol. player_season_np_shots_90: Tiros (sin incluir penales) por 90 minutos. Mide el volumen y la intención de un jugador de finalizar jugadas. player_season_np_xg_90 (Non-Penalty Expected Goals): La calidad de las oportunidades de tiro que un jugador genera para sí mismo. Un np_xg_90 alto indica que el jugador se posiciona constantemente en lugares de alta probabilidad de gol.   player_season_key_passes_90: Pases que resultan directamente en un tiro de un compañero. Es la métrica clásica para medir el volumen de creación de oportunidades. player_season_xa_90 (Expected Assists): Mide la calidad de las oportunidades que un jugador crea para sus compañeros. Un xa_90 alto sugiere que un jugador no solo da el último pase, sino que lo hace en situaciones muy ventajosas.   player_season_touches_inside_box_90: Toques de balón dentro del área rival. Es un excelente indicador de la presencia ofensiva y peligrosidad de un jugador, especialmente para delanteros y mediocampistas ofensivos. 3. Acciones Defensivas Esta categoría define el estilo y la contribución de un jugador sin la posesión del balón. ¿Es un jugador que presiona alto, un recuperador posicional o un especialista en duelos aéreos? player_season_pressures_90: Presiones aplicadas al rival por 90 minutos. Es la métrica fundamental para medir la intensidad y la voluntad de un jugador para participar en la fase defensiva alta.   player_season_tackles_90: Entradas exitosas. La métrica tradicional para medir la capacidad de un jugador para ganar el balón en duelos directos. player_season_interceptions_90: Intercepciones. Mide la inteligencia posicional y la capacidad de un jugador para leer el juego y cortar líneas de pase. player_season_aerial_wins_90: Duelos aéreos ganados. Esencial para perfilar defensores centrales y delanteros objetivo. player_season_padj_tackles_and_interceptions_90: Entradas e intercepciones ajustadas por posesión. Esta es una métrica avanzada importante. Normaliza las acciones defensivas según la cantidad de oportunidades que tiene un jugador para realizarlas (un jugador en un equipo con 30% de posesión tendrá más oportunidades de defender que uno en un equipo con 70%). Esto permite una comparación más justa.   4. Conducción y Dribles Define la capacidad de un jugador para progresar con el balón en los pies, en lugar de pasarlo. player_season_dribbles_90: Dribles completados con éxito. Mide la habilidad de un jugador en el 1 contra 1. player_season_carries_90: Conducciones de balón. Mide la frecuencia con la que un jugador controla y mueve el balón con los pies.   player_season_deep_progressions_90: Pases y conducciones hacia el último tercio del campo. Es una métrica compuesta excelente que captura la capacidad total de un jugador para hacer avanzar el juego hacia zonas peligrosas, ya sea pasando o corriendo con el balón.   ¿Por qué esta selección? Esta selección de ~15-20 variables te proporciona una visión de 360 grados del comportamiento de un jugador en el campo. No se enfoca solo en resultados (goles, asistencias), sino en los procesos que conducen a ellos. Estudios sobre perfilado de jugadores mediante PCA y clustering han demostrado que este tipo de métricas son las que mejor logran separar a los jugadores en roles funcionales distintos, como "mediocampista recuperador", "extremo creativo" o "delantero de área".   Al construir tu tabla de jugadores x métricas con estas variables, tendrás la materia prima perfecta para aplicar PCA. El análisis te revelará los "ejes de estilo" fundamentales en la Liga MX (por ejemplo, un eje podría ser "Progresión vs. Retención" y otro "Creación vs. Defensa"), permitiéndote visualizar a cada jugador en un mapa y medir objetivamente su similitud estilística.

In [26]:
import pandas as pd
import numpy as np
from statsbombpy import sb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from numpy.linalg import norm
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from collections import Counter
from sklearn.model_selection import KFold  # lo usa select_pca_k(method="cv")
from IPython.display import display        # opcional para imprimir tablas bonitas


#creds = {"user": "itam_hackathon@hudl.com", "passwd": "pGwIprel"}

# El Motor de Similitud de Jugadores

In [27]:
# Tus dicts (ya los tienes):
'''
partidos = {
    (317, '2024/2025'): sb.matches(competition_id=73, season_id=317, creds=creds),
    (281, '2023/2024'): sb.matches(competition_id=73, season_id=281, creds=creds),
    (235, '2022/2023'): sb.matches(competition_id=73, season_id=235, creds=creds),
    (108, '2021/2022'): sb.matches(competition_id=73, season_id=108, creds=creds)
}

equipos = {
    (317, '2024/2025'): partidos[(317, '2024/2025')]['home_team'].unique().tolist(),
    (281, '2023/2024'): partidos[(281, '2023/2024')]['home_team'].unique().tolist(),
    (235, '2022/2023'): partidos[(235, '2022/2023')]['home_team'].unique().tolist(),
    (108, '2021/2022'): partidos[(108, '2021/2022')]['home_team'].unique().tolist()
}

statsJugadorPorTemporada = {
    (317, '2024/2025'): sb.player_season_stats(competition_id=73, season_id=317, creds=creds),
    (281, '2023/2024'): sb.player_season_stats(competition_id=73, season_id=281, creds=creds),
    (235, '2022/2023'): sb.player_season_stats(competition_id=73, season_id=235, creds=creds),
    (108, '2021/2022'): sb.player_season_stats(competition_id=73, season_id=108, creds=creds)
}
'''

"\npartidos = {\n    (317, '2024/2025'): sb.matches(competition_id=73, season_id=317, creds=creds),\n    (281, '2023/2024'): sb.matches(competition_id=73, season_id=281, creds=creds),\n    (235, '2022/2023'): sb.matches(competition_id=73, season_id=235, creds=creds),\n    (108, '2021/2022'): sb.matches(competition_id=73, season_id=108, creds=creds)\n}\n\nequipos = {\n    (317, '2024/2025'): partidos[(317, '2024/2025')]['home_team'].unique().tolist(),\n    (281, '2023/2024'): partidos[(281, '2023/2024')]['home_team'].unique().tolist(),\n    (235, '2022/2023'): partidos[(235, '2022/2023')]['home_team'].unique().tolist(),\n    (108, '2021/2022'): partidos[(108, '2021/2022')]['home_team'].unique().tolist()\n}\n\nstatsJugadorPorTemporada = {\n    (317, '2024/2025'): sb.player_season_stats(competition_id=73, season_id=317, creds=creds),\n    (281, '2023/2024'): sb.player_season_stats(competition_id=73, season_id=281, creds=creds),\n    (235, '2022/2023'): sb.player_season_stats(competition_i

In [28]:
import sys, site, platform
print("PYTHON:", sys.executable)
print("VERSION:", sys.version)
print("SITE-PACKAGES:", site.getsitepackages() if hasattr(site, "getsitepackages") else site.getusersitepackages())


PYTHON: c:\ProgramData\Anaconda3\python.exe
VERSION: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
SITE-PACKAGES: ['c:\\ProgramData\\Anaconda3', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages']


In [29]:
import sys
!"{sys.executable}" -m pip install --upgrade pip
!"{sys.executable}" -m pip install pyarrow


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [30]:
import pyarrow
print("PyArrow version:", pyarrow.__version__)


PyArrow version: 21.0.0


In [31]:
ENGINE = "pyarrow"
SEASONS = [(317,"2024/2025"), (281,"2023/2024"), (235,"2022/2023"), (108,"2021/2022")]

# Jugadores
statsJugadorPorTemporada = {
    (sid, sname): pd.read_parquet(f"../data/Jugadores/player_season_stats_{sid}.parquet", engine=ENGINE)
    for (sid, sname) in SEASONS
}

# Partidos
partidos = {
    (sid, sname): pd.read_parquet(f"../data/Partidos/matches_competition_73_season_{sid}.parquet", engine=ENGINE)
    for (sid, sname) in SEASONS
}

equipos = {
    (sid, sname): partidos[(sid, sname)]["home_team"].dropna().unique().tolist()
    for (sid, sname) in SEASONS
}


In [32]:
METRICS = [
    "player_season_passing_ratio", "player_season_pass_length",
    "player_season_lbp_completed_90", "player_season_op_passes_into_box_90",
    "player_season_np_shots_90", "player_season_np_xg_90",
    "player_season_key_passes_90", "player_season_xa_90",
    "player_season_touches_inside_box_90",
    "player_season_pressures_90", "player_season_tackles_90",
    "player_season_interceptions_90", "player_season_aerial_wins_90",
    "player_season_padj_tackles_and_interceptions_90",
    "player_season_dribbles_90", "player_season_carries_90",
    "player_season_deep_progressions_90",
]
ID_COLS = ["player_id","player_name","team_id","team_name","competition_id","season_id","season_name"]
MINUTES_CANDIDATES = ["player_season_minutes","minutes","player_minutes","player_season_360_minutes"]

def _first_existing(colnames, candidates):
    for c in candidates:
        if c in colnames: return c
    return None

def concat_player_season_stats(statsJugadorPorTemporada, competition_id=73):
    frames = []
    for (season_id, season_name), df in statsJugadorPorTemporada.items():
        tmp = df.copy()
        if "competition_id" not in tmp.columns: tmp["competition_id"] = competition_id
        if "season_id" not in tmp.columns: tmp["season_id"] = season_id
        if "season_name" not in tmp.columns: tmp["season_name"] = season_name
        rename_map = {}
        if "name" in tmp.columns and "player_name" not in tmp.columns: rename_map["name"] = "player_name"
        if "team" in tmp.columns and "team_name" not in tmp.columns: rename_map["team"] = "team_name"
        if "teamId" in tmp.columns and "team_id" not in tmp.columns: rename_map["teamId"] = "team_id"
        if "playerId" in tmp.columns and "player_id" not in tmp.columns: rename_map["playerId"] = "player_id"
        if rename_map: tmp = tmp.rename(columns=rename_map)
        frames.append(tmp)
    if not frames: return pd.DataFrame()
    all_stats = pd.concat(frames, axis=0, ignore_index=True)
    if "player_season_lbp_completed_90" not in all_stats.columns:
        all_stats["player_season_lbp_completed_90"] = 0.0
    for c in ["team_id","team_name"]:
        if c not in all_stats.columns: all_stats[c] = np.nan
    for m in METRICS:
        if m in all_stats.columns: all_stats[m] = pd.to_numeric(all_stats[m], errors="coerce")
        else: all_stats[m] = np.nan
    return all_stats

def filter_by_minutes(df, min_minutes=900):
    min_col = _first_existing(df.columns, MINUTES_CANDIDATES)
    if min_col is None: return df.copy()
    return df[df[min_col] >= min_minutes].copy()

'''
def build_fingerprint_df(statsJugadorPorTemporada, competition_id=73, min_minutes=900,
                         n_components=3, group_standardization=("season_id",)):
    df = concat_player_season_stats(statsJugadorPorTemporada, competition_id=competition_id)
    df = filter_by_minutes(df, min_minutes=min_minutes)
    df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)
    Z_parts, scaler_by_group = [], {}
    gcols = list(group_standardization) if group_standardization else []
    if gcols:
        for gkey, gdf in df.groupby(gcols, dropna=False):
            X = gdf[METRICS].copy()
            med = X.median(numeric_only=True)
            X = X.fillna(med)
            scaler = StandardScaler()
            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index)
            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)
            Z_parts.append(gZ)
            scaler_by_group[gkey if isinstance(gkey, tuple) else (gkey,)] = scaler
        Zall = pd.concat(Z_parts, axis=0, ignore_index=True)
    else:
        X = df[METRICS].copy()
        med = X.median(numeric_only=True)
        X = X.fillna(med)
        scaler = StandardScaler()
        Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=df.index)
        Zall = pd.concat([df[ID_COLS], Xz.add_prefix("z_")], axis=1)
        scaler_by_group[("GLOBAL",)] = scaler
    Xz_all = Zall[[f"z_{m}" for m in METRICS]].values
    pca = PCA(n_components=n_components, random_state=42).fit(Xz_all)
    PCs = pca.transform(Xz_all)
    for i in range(n_components):
        Zall[f"PC{i+1}"] = PCs[:, i]
    return Zall, pca, scaler_by_group
'''

'\ndef build_fingerprint_df(statsJugadorPorTemporada, competition_id=73, min_minutes=900,\n                         n_components=3, group_standardization=("season_id",)):\n    df = concat_player_season_stats(statsJugadorPorTemporada, competition_id=competition_id)\n    df = filter_by_minutes(df, min_minutes=min_minutes)\n    df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)\n    Z_parts, scaler_by_group = [], {}\n    gcols = list(group_standardization) if group_standardization else []\n    if gcols:\n        for gkey, gdf in df.groupby(gcols, dropna=False):\n            X = gdf[METRICS].copy()\n            med = X.median(numeric_only=True)\n            X = X.fillna(med)\n            scaler = StandardScaler()\n            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index)\n            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)\n            Z_parts.append(gZ)\n            scaler_by_group[gkey if isinstance(gke

In [33]:
# === CONST: sinónimos de portero (normalizaremos a minúsculas) ===
GK_SYNONYMS = {"goalkeeper", "gk", "portero", "arquero", "guardameta", "goal keeper"}

# Normaliza etiquetas de posición (para cubrir "Goalkeeper", "GK", etc.)
def _norm_pos(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("-", " ").replace("_", " ")
    return s

def attach_positions(df_stats: pd.DataFrame, pos_csv_path: str) -> pd.DataFrame:
    """
    Une (season_id, player_id) con posiciones desde tu CSV externo.
    Espera columnas: season_id, player_id, position
    Crea columna 'position_norm' normalizada a minúsculas.
    """
    pos = pd.read_csv(pos_csv_path, usecols=["season_id", "player_id", "position"])
    pos["position_norm"] = pos["position"].map(_norm_pos)
    out = df_stats.merge(
        pos[["season_id", "player_id", "position_norm"]],
        on=["season_id", "player_id"],
        how="left"
    )
    return out

def drop_goalkeepers(df_stats_with_pos: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina SOLO si sabemos que es portero (position_norm ∈ GK_SYNONYMS).
    Si viene NaN (desconocido), LO DEJAMOS.
    """
    return df_stats_with_pos.loc[~df_stats_with_pos["position_norm"].isin(GK_SYNONYMS)].copy()

# ===== Builder: igual que tu build_fingerprint_df_autoK, pero parte de un DF ya filtrado =====
def build_fingerprint_df_autoK_FROM_DF(df_input: pd.DataFrame,
                                       k_method="parallel",
                                       variance_threshold=0.85,
                                       group_standardization=("season_id",),
                                       random_state=42):
    """
    Construye Zall (z-scores + PCA auto-K) tomando un DataFrame YA FILTRADO (sin porteros).
    Respeta tus METRICS/ID_COLS y la normalización por grupo (season_id).
    """
    # 1) Copia defensiva y asegura columnas
    df = df_input.copy()
    for c in ID_COLS:
        if c not in df.columns:
            df[c] = np.nan
    for m in METRICS:
        if m not in df.columns:
            df[m] = np.nan
        else:
            df[m] = pd.to_numeric(df[m], errors="coerce")

    # 2) Relleno específico (LBP si no existe en ciertas temporadas)
    if "player_season_lbp_completed_90" in df.columns:
        df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)

    # 3) Z-score por grupo (p. ej., por temporada)
    Z_parts, scaler_by_group = [], {}
    gcols = list(group_standardization) if group_standardization else []
    if gcols:
        for gkey, gdf in df.groupby(gcols, dropna=False):
            X = gdf[METRICS].copy()
            med = X.median(numeric_only=True)
            X = X.fillna(med)
            scaler = StandardScaler()
            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index)
            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)
            Z_parts.append(gZ)
            scaler_by_group[gkey if isinstance(gkey, tuple) else (gkey,)] = scaler
        Zall_new = pd.concat(Z_parts, ignore_index=True)
    else:
        X = df[METRICS].copy()
        med = X.median(numeric_only=True)
        X = X.fillna(med)
        scaler = StandardScaler()
        Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=df.index)
        Zall_new = pd.concat([df[ID_COLS], Xz.add_prefix("z_")], axis=1)
        scaler_by_group = {("GLOBAL",): scaler}

    # 4) PCA con K automático (usa tu select_pca_k existente)
    Xz_all = Zall_new[[f"z_{m}" for m in METRICS]].to_numpy()
    k, info = select_pca_k(Xz_all, method=k_method, variance_threshold=variance_threshold, random_state=random_state)
    pca = PCA(n_components=k, random_state=random_state).fit(Xz_all)
    PCs = pca.transform(Xz_all)
    for i in range(k):
        Zall_new[f"PC{i+1}"] = PCs[:, i]

    return Zall_new, pca, scaler_by_group, info


In [34]:
# Determina automáticamente cuántos compenentes del PCA usar o conservar y recibe el método que quiera usar 
# el usuario, nosotros lo estamos poniendo con el método paralell, que ajusta automáticamente el número de 
# componentes basado en datos
def select_pca_k(Xz, method, variance_threshold=0.85,
                 cv_folds=5, random_state=42, n_iter_parallel=200):
    n_samples, n_features = Xz.shape  # Obtiene dimensiones de la matriz: filas=muestras, columnas=métricas
    max_k = min(n_samples, n_features)  # El máximo k posible es el menor entre muestras y métricas

    pca_full = PCA(n_components=max_k, random_state=random_state).fit(Xz)  # Ajusta PCA con el máximo de componentes
    eigvals = pca_full.explained_variance_  # Autovalores: varianza explicada por cada componente
    prop = pca_full.explained_variance_ratio_  # Proporción de varianza explicada por cada componente
    cumprop = np.cumsum(prop)  # Varianza acumulada a medida que agregas componentes

    if method == "variance":  # Si eliges el criterio de varianza acumulada
        k = int(np.searchsorted(cumprop, variance_threshold) + 1)  # Primer k cuyo acumulado supera el umbral

    elif method == "parallel":  # Si usas Parallel Analysis (Horn)
        rng = np.random.default_rng(random_state)  # Generador de aleatoriedad reproducible
        rand_eigs = np.zeros((n_iter_parallel, len(eigvals)))  # Matriz para guardar autovalores de datos aleatorios
        for b in range(n_iter_parallel):  # Repite simulaciones aleatorias n_iter_parallel veces
            Xb = np.empty_like(Xz)  # Crea matriz vacía con misma forma que Xz
            for j in range(n_features):  # Recorre columnas para romper correlaciones
                Xb[:, j] = rng.permutation(Xz[:, j])  # Baraja la columna j (conserva distribución marginal)
            pca_b = PCA(n_components=max_k, random_state=random_state).fit(Xb)  # PCA sobre datos “sin estructura”
            rand_eigs[b, :len(pca_b.explained_variance_)] = pca_b.explained_variance_  # Guarda autovalores aleatorios
        mean_rand = rand_eigs.mean(axis=0)  # Promedio de autovalores aleatorios por componente
        k = int(np.sum(eigvals > mean_rand)) or 1  # Cuenta cuántos componentes reales superan al azar; al menos 1

    elif method == "broken_stick":  # Si usas el modelo teórico Broken-Stick
        H = np.array([np.sum(1.0/np.arange(j, n_features+1)) for j in range(1, n_features+1)])  # Sumas armónicas
        bs = H / H[0]  # Proporciones esperadas del modelo (normaliza por la primera suma)
        k = int(np.sum(prop >= bs[:len(prop)])) or 1  # Conserva componentes cuya proporción supera la línea teórica

    elif method == "cv":  # Si usas validación cruzada con error de reconstrucción
        kf = KFold(n_splits=cv_folds, shuffle=True, random_state=random_state)  # Define particiones KFold reproducibles
        mse_by_k = []  # Lista para guardar (k, mse_promedio)
        for k_try in range(1, max_k+1):  # Prueba k desde 1 hasta max_k
            mses = []  # Acumula errores por fold para este k
            pca_k = PCA(n_components=k_try, random_state=random_state)  # PCA con k_try componentes
            for tr, te in kf.split(Xz):  # Para cada split de entrenamiento/prueba
                pca_k.fit(Xz[tr])  # Ajusta PCA en el conjunto de entrenamiento
                X_te_proj = pca_k.inverse_transform(pca_k.transform(Xz[te]))  # Reconstruye el conjunto de prueba
                mses.append(np.mean((Xz[te] - X_te_proj)**2))  # Calcula MSE entre original y reconstruido
            mse_by_k.append((k_try, float(np.mean(mses))))  # Guarda el MSE promedio para este k
        k = min(mse_by_k, key=lambda t: t[1])[0]  # Elige el k con menor MSE promedio

    else:  # Si el método no es válido
        raise ValueError("method debe ser: 'variance' | 'parallel' | 'broken_stick' | 'cv'")  # Mensaje de error claro

    return int(k), {  # Devuelve k y un diccionario con detalles del ajuste
        "eigvals": eigvals,  # Autovalores reales (útil para gráficos/diagnóstico)
        "prop": prop,  # Proporción de varianza por componente
        "cumprop": cumprop,  # Varianza acumulada por número de componentes
        "selected_k": int(k),  # Número de componentes seleccionado
        "method": method  # Método utilizado para seleccionar k
    }

In [35]:
# Arma el Zall y aplica PCA con k óptimo
def build_fingerprint_df_autoK(statsJugadorPorTemporada, competition_id=73, min_minutes=900,
                               k_method="parallel", variance_threshold=0.85,
                               group_standardization=("season_id",), random_state=42):
    df = concat_player_season_stats(statsJugadorPorTemporada, competition_id=competition_id)  # Une temporadas en un DF
    df = filter_by_minutes(df, min_minutes=min_minutes)  # Filtra jugadores con pocos minutos
    df["player_season_lbp_completed_90"] = df["player_season_lbp_completed_90"].fillna(0)  # Si falta LBP, pon 0

    for c in ID_COLS:  # Recorre las columnas de identificación mínimas requeridas
        if c not in df.columns:  # Si alguna no existe en el DF original
            df[c] = np.nan  # Créala como NaN para evitar errores posteriores

    Z_parts, scaler_by_group = [], {}  # Inicializa lista de partes normalizadas y dict de escaladores por grupo
    gcols = list(group_standardization) if group_standardization else []  # Determina columnas de agrupación
    if gcols:  # Si hay agrupación (p.ej., por temporada)
        for gkey, gdf in df.groupby(gcols, dropna=False):  # Itera por grupo (temporada, o comp+temporada)
            X = gdf[METRICS].copy()  # Toma solo métricas
            med = X.median(numeric_only=True)  # Calcula medianas por métrica (para imputar)
            X = X.fillna(med)  # Imputa NaN con mediana (robusto a outliers)
            scaler = StandardScaler()  # Crea estandarizador para z-score
            Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=gdf.index)  # Aplica z-score al grupo
            gZ = pd.concat([gdf[ID_COLS], Xz.add_prefix("z_")], axis=1)  # Junta IDs con z_métricas
            Z_parts.append(gZ)  # Agrega el bloque normalizado a la lista
            scaler_by_group[gkey if isinstance(gkey, tuple) else (gkey,)] = scaler  # Guarda el scaler del grupo
        Zall = pd.concat(Z_parts, axis=0, ignore_index=True)  # Concatena todos los grupos normalizados
    else:  # Si no hay agrupación (normalización global)
        X = df[METRICS].copy()  # Toma métricas completas
        med = X.median(numeric_only=True)  # Medianas para imputar
        X = X.fillna(med)  # Imputa NaN
        scaler = StandardScaler()  # Estandarizador global
        Xz = pd.DataFrame(scaler.fit_transform(X), columns=METRICS, index=df.index)  # z-score global
        Zall = pd.concat([df[ID_COLS], Xz.add_prefix("z_")], axis=1)  # Concatena IDs + z_métricas
        scaler_by_group[("GLOBAL",)] = scaler  # Guarda el scaler global

    Xz_all = Zall[[f"z_{m}" for m in METRICS]].to_numpy()  # Extrae matriz numpy solo con z_métricas para PCA
    k, info = select_pca_k(  # Llama al selector automático para decidir el número óptimo de componentes
        Xz_all,
        method=k_method,  # Método elegido (por defecto 'parallel')
        variance_threshold=variance_threshold,  # Umbral si usaras 'variance'
        random_state=random_state  # Semilla para reproducibilidad
    )

    pca = PCA(n_components=k, random_state=random_state).fit(Xz_all)  # Ajusta PCA con k óptimo
    PCs = pca.transform(Xz_all)  # Proyecta los datos a los k componentes principales
    for i in range(k):  # Para cada componente calculado
        Zall[f"PC{i+1}"] = PCs[:, i]  # Crea columna PCi en Zall con el valor del componente para cada fila

    return Zall, pca, scaler_by_group, info  # Devuelve DF con PCs, el modelo PCA, escaladores y detalles de k

In [36]:
def similares(Zall, player_id, k=10):
    zcols = [c for c in Zall.columns if c.startswith("z_")]
    row = Zall.loc[Zall["player_id"]==player_id, zcols]
    if row.empty: raise ValueError(f"player_id {player_id} no encontrado.")
    v = row.to_numpy()[0]
    M = Zall[zcols].to_numpy()
    cos = (M @ v) / (norm(M,axis=1)*norm(v))
    out = Zall[["player_id","player_name","team_id","team_name","season_id","season_name"]].copy()
    out["cosine"] = cos
    return out[out["player_id"]!=player_id].nlargest(k, "cosine")

def get_team_members(statsJugadorPorTemporada, season_id, team_id):
    keys = [k for k in statsJugadorPorTemporada.keys() if k[0]==season_id]
    if not keys: raise ValueError(f"No hay stats para season_id={season_id}.")
    key = keys[0]
    df = statsJugadorPorTemporada[key].copy()
    rename_map = {}
    if "name" in df.columns and "player_name" not in df.columns: rename_map["name"] = "player_name"
    if "team" in df.columns and "team_name" not in df.columns: rename_map["team"] = "team_name"
    if "teamId" in df.columns and "team_id" not in df.columns: rename_map["teamId"] = "team_id"
    if "playerId" in df.columns and "player_id" not in df.columns: rename_map["playerId"] = "player_id"
    if rename_map: df = df.rename(columns=rename_map)
    if "team_id" not in df.columns: raise ValueError("No se encontró 'team_id' en player_season_stats.")
    roster = df[df["team_id"]==team_id][["player_id","player_name","team_id","team_name"]].drop_duplicates()
    return roster.sort_values("player_name").reset_index(drop=True)

In [37]:
# 0) Ruta a tu CSV de posiciones (usa r"" en Windows para backslashes)
pos_path = r"tpi_player_season_ligamx.csv"

# 1) Une temporadas como siempre
all_stats = concat_player_season_stats(statsJugadorPorTemporada, competition_id=73)

# 2) Adjunta posiciones desde tu CSV y ELIMINA PORTEROS
all_stats = attach_positions(all_stats, pos_path)
all_stats_no_gk = drop_goalkeepers(all_stats)

# (Opcional) Verifica cuántos eliminaste
n_before = len(all_stats)
n_after  = len(all_stats_no_gk)
print(f"Filas totales con posición: {n_before} | Sin porteros: {n_after} | Eliminados: {n_before - n_after}")

# 3) Filtro de minutos (como ya hacías)
all_stats_no_gk = filter_by_minutes(all_stats_no_gk, min_minutes=900)

# 4) Construye Zall DESDE este DF ya filtrado (sin porteros)
Zall, pca, scalers, kinfo = build_fingerprint_df_autoK_FROM_DF(
    all_stats_no_gk,
    k_method="parallel",
    variance_threshold=0.85,
    group_standardization=("season_id",),
    random_state=42
)

print("Filas (jugadores-temporadas) Sin porteros y con minutos:", len(Zall))
print("Varianza explicada PCA:", pca.explained_variance_ratio_)
Zall.head()


Filas totales con posición: 2820 | Sin porteros: 2706 | Eliminados: 114
Filas (jugadores-temporadas) Sin porteros y con minutos: 1508
Varianza explicada PCA: [0.39125228 0.26336449 0.08289817]


,player_id,player_name,team_id,team_name,competition_id,season_id,season_name,z_player_season_passing_ratio,z_player_season_pass_length,z_player_season_lbp_completed_90,...,z_player_season_tackles_90,z_player_season_interceptions_90,z_player_season_aerial_wins_90,z_player_season_padj_tackles_and_interceptions_90,z_player_season_dribbles_90,z_player_season_carries_90,z_player_season_deep_progressions_90,PC1,PC2,PC3
0,13034,Sebastián Saucedo Mondragón,1301,Pumas UNAM,73,108,2021/2022,-0.469687,-0.193116,-0.772879,...,-0.318935,-0.830877,-1.409879,-0.908478,1.282707,0.635200,-0.094918,2.471902,1.712512,0.198077
1,30458,Alejandro Zendejas Saavedra,1221,Necaxa,73,108,2021/2022,0.032157,-0.044561,0.214481,...,0.739356,0.708783,-0.596587,0.707159,0.675471,0.751720,0.952881,0.493582,2.376083,-1.325327
2,30458,Alejandro Zendejas Saavedra,1221,Necaxa,73,108,2021/2022,0.032157,-0.044561,0.214481,...,0.739356,0.708783,-0.596587,0.707159,0.675471,0.751720,0.952881,0.493582,2.376083,-1.325327
3,28529,Fernando David Arce Juárez,1291,Juárez,73,108,2021/2022,1.080911,0.086801,-0.060917,...,1.008524,1.947622,-0.429170,1.237603,-0.514283,0.061636,-0.793268,-2.271331,-0.271488,-2.384620
4,30458,Alejandro Zendejas Saavedra,1229,América,73,108,2021/2022,-0.025801,-0.925593,-0.314916,...,0.311967,-0.110381,-0.889628,0.088317,0.690732,0.579485,0.192812,1.892418,1.851183,-0.576871


In [38]:
player_id_objetivo = 13034  # cambia por un id real que veas en Zall["player_id"]
sim_df = similares(Zall, player_id=player_id_objetivo, k=10)
sim_df


,player_id,player_name,team_id,team_name,season_id,season_name,cosine
374,5637,Edgar Yoel Bárcenas Herrera,2773,Mazatlán,235,2022/2023,0.898391
1351,406131,Owen de Jesús González Ojeda,1298,Pachuca,317,2024/2025,0.892668
246,28418,Ían Jairo Misael Torres Ramírez,1296,Atlas,108,2021/2022,0.889027
247,28418,Ían Jairo Misael Torres Ramírez,1296,Atlas,108,2021/2022,0.889027
306,28528,Jean David Meneses Villarroel,1302,León,108,2021/2022,0.883914
307,28528,Jean David Meneses Villarroel,1302,León,108,2021/2022,0.883914
81,27899,Brian Avelino Lozano Aparicio,1299,Santos Laguna,108,2021/2022,0.881104
770,27228,Kevin Andrés Velasco Bonilla,1226,Puebla,281,2023/2024,0.874103
30,27356,Nicolás Benedetti Roa,2773,Mazatlán,108,2021/2022,0.867994
1147,27356,Nicolás Benedetti Roa,2773,Mazatlán,317,2024/2025,0.847853


In [39]:
season_id = 317  # 2024/2025
team_id = 1299   # pon el id real del club

roster = get_team_members(statsJugadorPorTemporada, season_id=season_id, team_id=team_id)
roster


,player_id,player_name,team_id,team_name
0,29265,Alan Jhosué Cervantes Martín del Campo,1299,Santos Laguna
1,37591,Aldo López Vargas,1299,Santos Laguna
2,5738,Anderson Santamaría Bardales,1299,Santos Laguna
3,7578,Anthony Rubén Lozano Colón,1299,Santos Laguna
4,50649,Bruno Amione,1299,Santos Laguna
5,45151,Bruno Barticciotto Di Bartolo,1299,Santos Laguna
6,32917,Carlos Acevedo López,1299,Santos Laguna
7,35719,Cristián Andrés Dájome Arboleda,1299,Santos Laguna
8,37878,Diego Javier Medina Vázquez,1299,Santos Laguna
9,26354,Edson Antonio Gutiérrez Moreno,1299,Santos Laguna


Ya tienes los PC1, PC2, PC3. El siguiente paso es darles un significado. Para que un scout o un director deportivo confíe en el modelo, no podemos decirles "el jugador A tiene un PC1 de 2.5". Debemos traducir eso a lenguaje de fútbol. ¿Cómo hacerlo?: Analizando los loadings del PCA (pca.components_). Esto te dirá qué métricas de tu lista METRICS tienen más peso en cada componente. Ejemplo de Interpretación: Podrías descubrir que: PC1 está positivamente correlacionado con np_shots_90, np_xg_90, touches_inside_box_90 y negativamente con tackles_90, interceptions_90. Conclusión: PC1 es un eje de "Aporte Ofensivo vs. Defensivo". PC2 está positivamente correlacionado con key_passes_90, xa_90, lbp_completed_90 y negativamente con pass_length. Conclusión: PC2 es un eje de "Creador de Juego vs. Jugador Directo/Vertical". Esto transforma tu mapa de puntos en un "mapa de estilos" interpretable.

In [40]:
# Matriz de loadings: cada fila = componente, cada columna = métrica
# Forma: (n_components, n_features)
loadings = pd.DataFrame(
    pca.components_,
    index=[f"PC{i+1}" for i in range(pca.n_components_)],
    columns=METRICS
)

# Correlación PC–métricas: corr(PCk, z_metrica)
# Usamos que PCs = Z * loadings^T (ortogonales), así que la correlación ~ loading * sqrt(var_exp)
# Para no depender de aproximaciones, la computamos empíricamente:
zcols = [f"z_{m}" for m in METRICS]
PCcols = [f"PC{i+1}" for i in range(pca.n_components_)]
corrs = pd.DataFrame(index=PCcols, columns=METRICS, dtype=float)

for pc in PCcols:
    for m, zm in zip(METRICS, zcols):
        corrs.loc[pc, m] = np.corrcoef(Zall[pc], Zall[zm])[0,1]

# Ordena y muestra top ± por PC
def top_features_for_pc(pc_name, k=6):
    s = corrs.loc[pc_name].dropna()
    top_pos = s.sort_values(ascending=False).head(k)
    top_neg = s.sort_values(ascending=True).head(k)
    return top_pos, top_neg

for pc in PCcols:
    pos, neg = top_features_for_pc(pc, k=6)
    print(f"\n=== {pc} ===")
    print("↑ Positivos (correlan +):")
    print(pos.to_string())
    print("↓ Negativos (correlan -):")
    print(neg.to_string())



=== PC1 ===
↑ Positivos (correlan +):
player_season_touches_inside_box_90    0.877612
player_season_np_shots_90              0.875949
player_season_np_xg_90                 0.848595
player_season_xa_90                    0.662341
player_season_key_passes_90            0.611243
player_season_dribbles_90              0.567114
↓ Negativos (correlan -):
player_season_interceptions_90                    -0.834294
player_season_padj_tackles_and_interceptions_90   -0.731360
player_season_pass_length                         -0.703202
player_season_passing_ratio                       -0.696164
player_season_lbp_completed_90                    -0.633464
player_season_tackles_90                          -0.527996

=== PC2 ===
↑ Positivos (correlan +):
player_season_deep_progressions_90     0.872202
player_season_carries_90               0.822621
player_season_op_passes_into_box_90    0.732748
player_season_key_passes_90            0.677449
player_season_xa_90                    0.572902
player_s

In [41]:
OFFENSIVE = {
    "player_season_np_shots_90",
    "player_season_np_xg_90",
    "player_season_touches_inside_box_90",
    "player_season_key_passes_90",
    "player_season_xa_90",
    "player_season_op_passes_into_box_90",
    "player_season_dribbles_90",
    "player_season_carries_90",
    "player_season_deep_progressions_90",
    "player_season_lbp_completed_90",
}

def orient_pc_signs(Zall, pca, corrs, offensive_set=OFFENSIVE):
    PCcols = [f"PC{i+1}" for i in range(pca.n_components_)]
    flipped = {}
    for i, pc in enumerate(PCcols):
        s = corrs.loc[pc, list(offensive_set & set(METRICS))].dropna()
        score = s.sum()
        if score < 0:
            # flip en Zall, corrs y loadings/pca
            Zall[pc] = -Zall[pc]
            corrs.loc[pc, :] = -corrs.loc[pc, :]
            pca.components_[i, :] = -pca.components_[i, :]
            flipped[pc] = True
        else:
            flipped[pc] = False
    return flipped

flipped_info = orient_pc_signs(Zall, pca, corrs, OFFENSIVE)
print("PCs volteados por convención ofensiva:", flipped_info)


PCs volteados por convención ofensiva: {'PC1': False, 'PC2': False, 'PC3': False}


In [42]:
LABEL_RULES = [
    {
        "name": "Aporte Ofensivo vs Defensivo",
        "positives": {"player_season_np_shots_90","player_season_np_xg_90",
                      "player_season_touches_inside_box_90"},
        "negatives": {"player_season_tackles_90","player_season_interceptions_90"}
    },
    {
        "name": "Creador Asociativo vs Juego Directo",
        "positives": {"player_season_key_passes_90","player_season_xa_90",
                      "player_season_lbp_completed_90","player_season_op_passes_into_box_90"},
        "negatives": {"player_season_pass_length"}
    },
    {
        "name": "Intensidad de Presión/Recuperación",
        "positives": {"player_season_pressures_90","player_season_tackles_90",
                      "player_season_interceptions_90","player_season_padj_tackles_and_interceptions_90"},
        "negatives": set()
    },
    {
        "name": "Progresión en Conducción (1v1)",
        "positives": {"player_season_dribbles_90","player_season_carries_90","player_season_deep_progressions_90"},
        "negatives": set()
    },
    {
        "name": "Dominio Aéreo",
        "positives": {"player_season_aerial_wins_90"},
        "negatives": set()
    }
]

def score_label_rule(pc_name, corrs, rule):
    pos = corrs.loc[pc_name, list(rule["positives"] & set(METRICS))].sum() if rule["positives"] else 0.0
    neg = -corrs.loc[pc_name, list(rule["negatives"] & set(METRICS))].sum() if rule["negatives"] else 0.0
    return pos + neg

def auto_label_components(corrs, rules=LABEL_RULES):
    labels = {}
    for pc in corrs.index:
        scores = [(score_label_rule(pc, corrs, r), r["name"]) for r in rules]
        scores.sort(reverse=True)
        best = scores[0]
        labels[pc] = {"label": best[1], "score": float(best[0])}
    return labels

auto_labels = auto_label_components(corrs, LABEL_RULES)
auto_labels


{'PC1': {'label': 'Aporte Ofensivo vs Defensivo', 'score': 3.9644460085537307},
 'PC2': {'label': 'Creador Asociativo vs Juego Directo',
  'score': 2.5695803604657743},
 'PC3': {'label': 'Aporte Ofensivo vs Defensivo', 'score': 0.7177892307578715}}

In [43]:
def pc_summary(pc, k=5):
    s = corrs.loc[pc].dropna()
    top_pos = s.sort_values(ascending=False).head(k)
    top_neg = s.sort_values(ascending=True).head(k)
    lab = auto_labels[pc]["label"]
    text = f"• {pc} — {lab}\n" \
           f"  Señales ↑: {', '.join([f'{m} ({top_pos[m]:+.2f})' for m in top_pos.index])}\n" \
           f"  Señales ↓: {', '.join([f'{m} ({top_neg[m]:+.2f})' for m in top_neg.index])}"
    return text

for pc in PCcols:
    print(pc_summary(pc, k=4))


• PC1 — Aporte Ofensivo vs Defensivo
  Señales ↑: player_season_touches_inside_box_90 (+0.88), player_season_np_shots_90 (+0.88), player_season_np_xg_90 (+0.85), player_season_xa_90 (+0.66)
  Señales ↓: player_season_interceptions_90 (-0.83), player_season_padj_tackles_and_interceptions_90 (-0.73), player_season_pass_length (-0.70), player_season_passing_ratio (-0.70)
• PC2 — Creador Asociativo vs Juego Directo
  Señales ↑: player_season_deep_progressions_90 (+0.87), player_season_carries_90 (+0.82), player_season_op_passes_into_box_90 (+0.73), player_season_key_passes_90 (+0.68)
  Señales ↓: player_season_aerial_wins_90 (-0.72), player_season_np_xg_90 (-0.23), player_season_touches_inside_box_90 (-0.22), player_season_pass_length (-0.03)
• PC3 — Aporte Ofensivo vs Defensivo
  Señales ↑: player_season_pass_length (+0.40), player_season_lbp_completed_90 (+0.25), player_season_carries_90 (+0.24), player_season_passing_ratio (+0.17)
  Señales ↓: player_season_pressures_90 (-0.73), player_

In [44]:
# Implementación simple de varimax
def varimax(Phi, gamma = 1.0, q = 20, tol = 1e-6):
    p,k = Phi.shape
    R = np.eye(k)
    d=0
    for i in range(q):
        d_old = d
        Lambda = Phi @ R
        u,s,vh = np.linalg.svd(Phi.T @ (Lambda**3 - (gamma/p) * Lambda @ np.diag(np.diag(Lambda.T @ Lambda))))
        R = u @ vh
        d = np.sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    return Phi @ R, R

# Aplica varimax a los loadings originales
L = pca.components_.T  # (features, components)
Lv, R = varimax(L)     # rotated loadings
rot_loadings = pd.DataFrame(Lv.T, index=PCcols, columns=METRICS)

# Recalcula correlaciones aproximadas usando Lv si quieres (o recomputa PCs rotados).


Ahora que tenemos una base sólida y componentes interpretables, el siguiente módulo a construir es el de Clustering para la Definición de Roles (K-Means).

El objetivo es usar tus componentes principales (PC1, PC2, PC3) para agrupar formalmente a los jugadores en arquetipos funcionales.

Determinar el Número Óptimo de Clústeres (k):

Utiliza el "método del codo" (Elbow Method). Consiste en ejecutar K-Means para un rango de valores de k (ej., de 2 a 15 clústeres) y graficar la inercia (suma de las distancias al cuadrado de cada punto a su centroide). El "codo" en la gráfica sugiere el número óptimo de clústeres, donde añadir más clústeres ya no aporta una mejora significativa.

Ejecutar K-Means y Asignar Roles:

Una vez que tengas el k óptimo, ejecuta el algoritmo K-Means sobre tus datos de componentes principales (Zall[['PC1', 'PC2', 'PC3']]).

Esto asignará a cada jugador-temporada en tu DataFrame Zall una etiqueta de clúster (ej., 0, 1, 2,... k-1). Esta etiqueta es su rol basado en datos.

Perfilar y Etiquetar los Roles Descubiertos:

Un clúster es solo un número. Para que sea útil, debemos darle un nombre. Para ello, agrupa tu DataFrame Zall por la nueva etiqueta de clúster y calcula la media de las métricas estandarizadas (z_...) para cada grupo.

Al analizar estos perfiles promedio, podrás darles nombres descriptivos. Por ejemplo:

Clúster 4: Podrías ver que tiene valores z_ muy altos para player_season_pressures_90 y player_season_tackles_90. Lo podrías etiquetar como "Mediocampista Recuperador".

Clúster 1: Podría tener valores z_ altos para player_season_dribbles_90 y player_season_crosses_90. Etiqueta: "Extremo Clásico".

El resultado final de este siguiente paso será un DataFrame Zall enriquecido con una columna de "Rol", y un entendimiento claro de lo que cada rol significa en términos estadísticos. Esto nos permitirá pasar de buscar jugadores "similares a X" a buscar "el mejor Mediocampista Recuperador disponible en el mercado".

In [45]:
PC_COLS = [f"PC{i+1}" for i in range(pca.n_components_)]

def _get_Xpcs(Zall, scale_pc=True, pc_cols=None):
    if pc_cols is None:
        pc_cols = [c for c in Zall.columns if c.startswith("PC")]
    X = Zall[pc_cols].to_numpy().copy()
    if scale_pc:
        X = (X - X.mean(axis=0)) / (X.std(axis=0, ddof=0) + 1e-12)
    return X



In [46]:
def auto_select_k(Zall, k_min=2, k_max=15, *, method="silhouette",
                  scale_pc=True, pc_cols=None, random_state=42, n_init=50):
    """
    Elige k automáticamente con uno de:
      - 'silhouette' (↑ mejor)
      - 'ch'         (Calinski–Harabasz ↑)
      - 'db'         (Davies–Bouldin ↓)
      - 'ensemble'   (votación entre silhouette, CH y DB; desempata con silhouette, luego CH, luego menor k)
    Devuelve: k_opt, justificación, diag(DataFrame con métricas por k)
    """
    X = _get_Xpcs(Zall, scale_pc=scale_pc, pc_cols=pc_cols)
    rows = []
    for k in range(k_min, k_max+1):
        km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init).fit(X)
        labels = km.labels_
        rows.append({
            "k": k,
            "inertia": float(km.inertia_),
            "silhouette": float(silhouette_score(X, labels)) if k > 1 else float("nan"),
            "ch": float(calinski_harabasz_score(X, labels)) if k > 1 else float("nan"),
            "db": float(davies_bouldin_score(X, labels)) if k > 1 else float("nan"),
        })
    diag = pd.DataFrame(rows)

    method = method.lower()
    if method == "silhouette":
        idx = diag["silhouette"].idxmax()
        k_opt = int(diag.loc[idx, "k"])
        just = f"Máximo Silhouette = {diag.loc[idx, 'silhouette']:.3f}"
    elif method == "ch":
        idx = diag["ch"].idxmax()
        k_opt = int(diag.loc[idx, "k"])
        just = f"Máximo Calinski–Harabasz = {diag.loc[idx, 'ch']:.1f}"
    elif method == "db":
        idx = diag["db"].idxmin()
        k_opt = int(diag.loc[idx, "k"])
        just = f"Mínimo Davies–Bouldin = {diag.loc[idx, 'db']:.3f}"
    elif method == "ensemble":
        # ganador por cada métrica
        k_sil = int(diag.loc[diag["silhouette"].idxmax(), "k"])
        k_ch  = int(diag.loc[diag["ch"].idxmax(), "k"])
        k_db  = int(diag.loc[diag["db"].idxmin(), "k"])
        votos = Counter([k_sil, k_ch, k_db])
        max_votos = max(votos.values())
        candidatos = sorted([k for k, v in votos.items() if v == max_votos])  # lista ordenada
        if len(candidatos) == 1:
            k_opt = candidatos[0]
        else:
            # desempates: mejor silhouette, luego CH, luego menor k
            best_sil_k = int(diag.loc[diag["silhouette"].idxmax(), "k"])
            if best_sil_k in candidatos:
                k_opt = best_sil_k
            else:
                best_ch_k = int(diag.loc[diag["ch"].idxmax(), "k"])
                if best_ch_k in candidatos:
                    k_opt = best_ch_k
                else:
                    k_opt = min(candidatos)
        just = f"Ensemble (votos: Sil={k_sil}, CH={k_ch}, DB={k_db}) → k={k_opt}"
    else:
        raise ValueError("method debe ser: 'silhouette' | 'ch' | 'db' | 'ensemble'")

    return k_opt, just, diag

In [47]:
def fit_kmeans_and_assign(Zall, k, *, scale_pc=True, colname=None, pc_cols=None, random_state=42, n_init=50):
    if colname is None:
        colname = f"cluster_k{k}"
    X = _get_Xpcs(Zall, scale_pc=scale_pc, pc_cols=pc_cols)
    km = KMeans(n_clusters=k, random_state=random_state, n_init=n_init)
    labels = km.fit_predict(X)
    Zall[colname] = labels
    return Zall, km

In [48]:
def profile_clusters(Zall, cluster_col):
    """
    Regresa:
      - prof_z: medias de todas las z_ métricas por clúster (para interpretar).
      - size: tamaño de cada clúster.
    """
    zcols = [f"z_{m}" for m in METRICS if f"z_{m}" in Zall.columns]
    grp = Zall.groupby(cluster_col, dropna=False)
    prof_z = grp[zcols].mean().sort_index()
    size = grp.size().rename("n")
    return prof_z, size


In [49]:
ROLE_RULES = [
    # Recuperación/Presión
    {
        "name_pos": "Mediocampista Recuperador",
        "pos": {"player_season_pressures_90","player_season_tackles_90",
                "player_season_interceptions_90","player_season_padj_tackles_and_interceptions_90"},
        "neg": set()
    },
    # Extremo desequilibrante
    {
        "name_pos": "Extremo Desequilibrante",
        "pos": {"player_season_dribbles_90","player_season_carries_90",
                "player_season_deep_progressions_90"},
        "neg": set()
    },
    # Creador Asociativo
    {
        "name_pos": "Creador Asociativo",
        "pos": {"player_season_key_passes_90","player_season_xa_90",
                "player_season_lbp_completed_90","player_season_op_passes_into_box_90"},
        "neg": {"player_season_pass_length"}
    },
    # Finalizador/9 de área
    {
        "name_pos": "Delantero de Área",
        "pos": {"player_season_np_shots_90","player_season_np_xg_90",
                "player_season_touches_inside_box_90"},
        "neg": set()
    },
    # Central dominante aéreo
    {
        "name_pos": "Defensa Central Aéreo",
        "pos": {"player_season_aerial_wins_90"},
        "neg": set()
    },
    # Lateral/progresión mixta
    {
        "name_pos": "Lateral Progresivo",
        "pos": {"player_season_deep_progressions_90","player_season_op_passes_into_box_90"},
        "neg": set()
    },
]

def score_rule(cluster_mean, rule):
    s_pos = cluster_mean[[f"z_{m}" for m in rule["pos"] if f"z_{m}" in cluster_mean.index]].sum()
    s_neg = -cluster_mean[[f"z_{m}" for m in rule["neg"] if f"z_{m}" in cluster_mean.index]].sum() if rule["neg"] else 0.0
    return float(s_pos + s_neg)

def auto_name_clusters(prof_z):
    """
    prof_z: DataFrame (cluster x z_metrics_mean)
    Devuelve dict {cluster_label: {'role': str, 'top_pos': list, 'top_neg': list}}
    """
    out = {}
    for cl in prof_z.index:
        row = prof_z.loc[cl]
        # ranking de reglas
        scored = [(score_rule(row, r), r["name_pos"]) for r in ROLE_RULES]
        scored.sort(reverse=True)
        role = scored[0][1]
        # features top para explicación
        s = row.sort_values(ascending=False)
        top_pos = s.head(5).index.tolist()
        top_neg = row.sort_values(ascending=True).head(5).index.tolist()
        out[cl] = {
            "role": role,
            "top_pos": top_pos,
            "top_neg": top_neg
        }
    return out


In [50]:
def cluster_and_label_roles_auto(Zall, *, k_min=2, k_max=15, method="silhouette",
                                 scale_pc=True, pc_cols=None, random_state=42, n_init=50):
    
    k_opt, just, diag = auto_select_k(
        Zall, k_min=k_min, k_max=k_max, method=method,
        scale_pc=scale_pc, pc_cols=pc_cols, random_state=random_state, n_init=n_init
    )
    cluster_col = f"cluster_k{k_opt}"
    Zall, km = fit_kmeans_and_assign(
        Zall, k_opt, scale_pc=scale_pc, colname=cluster_col,
        pc_cols=pc_cols, random_state=random_state, n_init=n_init
    )
    prof_z, size = profile_clusters(Zall, cluster_col)
    names = auto_name_clusters(prof_z)
    Zall["Rol"] = Zall[cluster_col].map({cl: info["role"] for cl, info in names.items()})
    return Zall, km, prof_z, size, names, k_opt, just, diag


In [51]:
# Selección automática de k y clustering + roles
Zall, km, prof_z, size, names, k_opt, just, diag = cluster_and_label_roles_auto(
    Zall,
    k_min=2, k_max=15,
    method="ensemble",      # o "silhouette", "ch", "db"
    scale_pc=True,
    pc_cols=PC_COLS,
    random_state=42, n_init=50
)
print("size: ",size)
print("hola")

# >>> AHORA sí: corre tu línea para ver los roles disponibles en la temporada 317
roles_disponibles = sorted(Zall.loc[Zall["season_id"]==317, "Rol"].dropna().unique())
print(roles_disponibles)


print(Zall["Rol"])
print("Tamaños por clúster:\n", size)
print("\nNombres asignados:")
for cl, info in names.items():
    print(f"  Cluster {cl}: {info['role']}  | Señales↑ {info['top_pos'][:3]}  | Señales↓ {info['top_neg'][:3]}")

# 3) Guarda resultados
Zall.to_csv(f"Zall_roles_k{k_opt}.csv", index=False)
prof_z.to_csv(f"perfil_z_k{k_opt}.csv")
size.to_frame().to_csv(f"cluster_sizes_k{k_opt}.csv")


size:  cluster_k4
0    488
1    216
2    437
3    367
Name: n, dtype: int64
hola
['Creador Asociativo', 'Defensa Central Aéreo', 'Delantero de Área', 'Mediocampista Recuperador']
0              Creador Asociativo
1       Mediocampista Recuperador
2       Mediocampista Recuperador
3       Mediocampista Recuperador
4              Creador Asociativo
                  ...            
1503           Creador Asociativo
1504           Creador Asociativo
1505           Creador Asociativo
1506           Creador Asociativo
1507           Creador Asociativo
Name: Rol, Length: 1508, dtype: object
Tamaños por clúster:
 cluster_k4
0    488
1    216
2    437
3    367
Name: n, dtype: int64

Nombres asignados:
  Cluster 0: Mediocampista Recuperador  | Señales↑ ['z_player_season_tackles_90', 'z_player_season_padj_tackles_and_interceptions_90', 'z_player_season_pressures_90']  | Señales↓ ['z_player_season_np_xg_90', 'z_player_season_touches_inside_box_90', 'z_player_season_np_shots_90']
  Cluster 1: Dela

In [52]:
print("Jugadores-temporada en Zall:", len(Zall))
print("¿Columna Rol existe?", "Rol" in Zall.columns)
print("Roles detectados:", sorted(Zall["Rol"].dropna().unique()))
print("Temporada 317 - conteo por Rol:")
print(Zall[Zall["season_id"]==317]["Rol"].value_counts())



Jugadores-temporada en Zall: 1508
¿Columna Rol existe? True
Roles detectados: ['Creador Asociativo', 'Defensa Central Aéreo', 'Delantero de Área', 'Mediocampista Recuperador']
Temporada 317 - conteo por Rol:
Defensa Central Aéreo        128
Mediocampista Recuperador    122
Creador Asociativo            89
Delantero de Área             50
Name: Rol, dtype: int64


In [53]:
print("Filas totales (jugador-temporada):", len(Zall))
print("Jugadores únicos:", Zall["player_id"].nunique())

Filas totales (jugador-temporada): 1508
Jugadores únicos: 536
